<h1 style="text-align: center;text-transform: uppercase;">Conversational Based Agent</h1>

<br>

In this project, you will build an end-to-end voice conversational agent, which can take a voice input audio line, and synthesize a response. The chatbot agent will be executed locally on your computer. 

<img style="width:550px; height:300px;" src="assets/intro.png">

This project consists of the following parts:
1. __Speech Recognition:__ <br>In this part, you will create a speech recognition that can convert your voice into a text format.<br><br>
2. __Chatbot:__ <br>This is the core of your conversational based agent. You will build a chatbot that will answer your questions. <br><br>
3. __Text to Speech:__ <br>After getting the answer from your chatbot, it should be converted into a voice format and that is what you should create in this part. <br><br>
4. __Finalize your Conversational Based Agent:__ <br>At the very end step, you will put everything together and create your Conversational Based Agent.

<br>

# 2. Chatbot

---


In this part, you will create a deep learning based conversational agent. This agent will be able to interact with users and understand their questions. More specifically, you will start with loading the dataset, cleaning and preprocessing them, and then you will feed them into a neural network for training.

<br>

### 2.2. Train the Seq2Seq Model

---

In this section, we will use an architecture called Sequence to Sequence (or Seq2Seq). This model is used since the length of the input sequence (question) does not match the length of the output sequence (answer). This model is consists of an encoder and a decoder.
- __Encoder:__ <br> In this part of the network, we take the input data and train on it. Then we pass the last state of the recurrent layer to decoder. <br><br>
- __Decoder:__ <br> In this part of the network, we take the last state in encoder’s last recurrent layer. Then we will use it as an initial state in decoder's first recurrent layer.

<br>

<img src="assets/encoder_decoder.png">

<br>

Let's start by importing all the necessary libraries in Keras.

In [119]:
# Import the libraries
import tensorflow.keras
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.activations import softmax
from tensorflow.keras.callbacks import ModelCheckpoint

Below you can play around with hyperparameters for improving the model's accuracy.

In [120]:
# Hyper parameters
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 1e-3

In the following block of code, you will implement the Encoder. You can follow the below steps for creating the encoder: 

1.   Create an input for the Encoder.
2.   Create an embedding layer.
3.   Create an LSTM layer which also returns the states.
4.   Get the hidden state (state h) and cell state (state c) inside a list.

In [126]:
### Encoder Input
embed_dim = 200
num_lstm = 200

# Input for encoder
encoder_inputs = Input(shape = (None, ), name='encoder_inputs')

# Embedding layer
# Why mask_zero = True? https://www.tensorflow.org/guide/keras/masking_and_padding
encoder_embedding = Embedding(input_dim = VOCAB_SIZE, output_dim = embed_dim, mask_zero = True, name='encoder_embedding')(encoder_inputs)

# LSTM layer (that returns states in addition to output)
encoder_outputs, state_h, state_c = LSTM(units = num_lstm, return_state = True, name='encoder_lstm')(encoder_embedding)

# Get the states for encoder
encoder_states = [state_h, state_c]

After creating your encoder, it's time to implement the decoder. You can follow the below steps for implementing the decoder:

1.   Create an input for the decoder.
2.   Create an embedding layer.
3.   Create an LSTM layer that returns states and sequences.
4.   Create a dense layer.
5.   Get the output.

In [128]:
### Decoder

# Input for decoder
decoder_inputs = Input(shape = (None,  ), name='decoder_inputs')

# Embedding layer
decoder_embedding = Embedding(input_dim = VOCAB_SIZE, output_dim = embed_dim , mask_zero = True, name='decoder_embedding')(decoder_inputs)

# LSTM layer (that returns states and sequences as well)
decoder_lstm = LSTM(units = num_lstm , return_state = True , return_sequences = True, name='decoder_lstm')

# Get the output of LSTM layer, using the initial states from the encoder
decoder_outputs, _, _ = decoder_lstm(inputs = decoder_embedding, initial_state = encoder_states)

# Dense layer
decoder_dense = Dense(units = VOCAB_SIZE, activation = softmax, name='output') 

# Get the output of Dense layer
output = decoder_dense(decoder_outputs)

Now that you have implemented the encoder and decoder. It's time to create your model which takes two inputs: encoder's input and decoder's input. Then it outputs the decoder's output.

In [129]:
# Create the model
model = Model([encoder_inputs, decoder_inputs], output)

In [130]:
# Compile the model
model.compile(optimizer = RMSprop(lr = LEARNING_RATE), loss = "categorical_crossentropy")

In [131]:
# Summary
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
encoder_embedding (Embedding)   (None, None, 200)    387800      encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_embedding (Embedding)   (None, None, 200)    387800      decoder_inputs[0][0]             
___________________________________________________________________________________________

In [ ]:
# Load all the arrays from storage
encoder_input_data = np.load("enc_in_data.npy")
decoder_input_data = np.load("dec_in_data.npy")
decoder_output_data = np.load("dec_out_data.npy")

In [132]:
# Train the model
model.fit(x = [encoder_input_data , decoder_input_data], 
          y = decoder_output_data, 
          batch_size = BATCH_SIZE, 
          epochs = EPOCHS) 

Train on 869 samples
Epoch 1/100
869/869 [==============================] - 9s 11ms/sample - loss: 1.4653
Epoch 2/100
869/869 [==============================] - 8s 9ms/sample - loss: 1.2848
Epoch 3/100
869/869 [==============================] - 7s 8ms/sample - loss: 1.2320
Epoch 4/100
869/869 [==============================] - 7s 8ms/sample - loss: 1.1936
Epoch 5/100
869/869 [==============================] - 7s 8ms/sample - loss: 1.1620
Epoch 6/100
869/869 [==============================] - 7s 8ms/sample - loss: 1.1305
Epoch 7/100
869/869 [==============================] - 7s 8ms/sample - loss: 1.1005
Epoch 8/100
869/869 [==============================] - 7s 8ms/sample - loss: 1.0700
Epoch 9/100
869/869 [==============================] - 7s 8ms/sample - loss: 1.0410
Epoch 10/100
869/869 [==============================] - 7s 8ms/sample - loss: 1.0121
Epoch 11/100
869/869 [==============================] - 7s 8ms/sample - loss: 0.9857
Epoch 12/100
869/869 [==============================

In [133]:
# Save the final model
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
model.save(filepath = f'saved_models/final_weight_{timestamp}.h5') 
print(f"Model Weight Saved to {final_weight_{timestamp}.h5}!")

Model Weight Saved!


In [70]:
# Load the final model
model.load_weights(f'saved_models/final_weight_2020-08-01-16-28-47.h5') 
print("Model Weight Loaded!")

Model Weight Loaded!
